# CNN Extractor + Regressor

### Constant

In [1]:
DATA_PATH = "../../data.nosync/subj01"
EXTRACTOR = "resnet50-imagenet1k-v2"
LAYER = "avgpool"

## Load data

In [2]:
from sklearn.model_selection import train_test_split
from src import dataset

feat, l_frmi, r_frmi = dataset.get_dataset(
    DATA_PATH, EXTRACTOR, LAYER, True)

# normalize
feat = (feat - feat.mean()) / feat.std()

X_train, X_test, l_fmri_train, l_fmri_test, r_fmri_train, r_fmri_test = train_test_split(feat, l_frmi, r_frmi, train_size=0.8)

print("X_train shape: {}".format(X_train.shape))
print("l_fmri_train shape: {}".format(l_fmri_train.shape))
print("r_fmri_train shape: {}".format(r_fmri_train.shape))

print()

print("X_test shape: {}".format(X_test.shape))
print("l_fmri_test shape: {}".format(l_fmri_test.shape))
print("r_fmri_test shape: {}".format(r_fmri_test.shape))


X_train shape: (7872, 2048)
l_fmri_train shape: (7872, 19004)
r_fmri_train shape: (7872, 20544)

X_test shape: (1969, 2048)
l_fmri_test shape: (1969, 19004)
r_fmri_test shape: (1969, 20544)


## Modelling

In [26]:
from sklearn.linear_model import Ridge

model_l = Ridge(alpha=2e4).fit(X=X_train, y=l_fmri_train)
y_pred_l = model_l.predict(X_test)

In [27]:
model_r = Ridge(alpha=2e4).fit(X=X_train, y=r_fmri_train)
y_pred_r = model_r.predict(X_test)

In [28]:
import numpy as np
from scipy.stats import pearsonr

def compute_perason(pred, target):

    corrcoef = list()
    for pred, target in zip(pred.T, target.T):

        s, _ = pearsonr(x=pred, y=target)
        corrcoef.append(s)

    return np.array(corrcoef)


lh_correlation = compute_perason(y_pred_l, l_fmri_test)
rh_correlation = compute_perason(y_pred_r, r_fmri_test)

## Visualization

### Dev Pearson's R

In [33]:
from src.visualize import histogram, box_plot

histogram(DATA_PATH, lh_correlation, rh_correlation, "Ridge, alpha=2e4, ResNet50-Imagenet1k-AvgPooling, Full Image", "./img/ridge_resnet50_full_img_alpha_2e4/")

/opt/miniconda3/envs/algonauts/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/opt/miniconda3/envs/algonauts/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



In [34]:
box_plot(DATA_PATH, lh_correlation, rh_correlation,
         "Ridge, alpha=2e4, ResNet50-Imagenet1k-AvgPooling, Full Image", "./img/ridge_resnet50_full_img_alpha_2e4/")


### Train Pearson's R

In [30]:
y_pred_l_train = model_l.predict(X_train)
y_pred_r_train = model_r.predict(X_train)

In [31]:
lh_correlation_train = compute_perason(y_pred_l_train, l_fmri_train)
rh_correlation_train = compute_perason(y_pred_r_train, r_fmri_train)

In [32]:
histogram(DATA_PATH, lh_correlation_train, rh_correlation_train,
          "Ridge, alpha=2e4, ResNet50-Imagenet1k-AvgPooling, Full Image, TRAIN", "./img/ridge_resnet50_full_img_alpha_2e4/")


/opt/miniconda3/envs/algonauts/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/opt/miniconda3/envs/algonauts/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



In [11]:
box_plot(DATA_PATH, lh_correlation_train, rh_correlation_train,
               "Ridge, alpha=2e4, ResNet50-Imagenet1k-AvgPooling, Full Image, TRAIN")
